* REmote DIctionary Server
* Open source, written in C.
* It supports string, hashes, lists, sets, sorted sets

* Fast in memory key-value store
* Scalable and fault tolerant, keeps data in memory and write to disk periodically. In case of failure last few seconds of key/value are lost.
* Useful for
    - Saving game state of million of online users
    - Leader board: real time user ranking
    - Session cache(store)
    - Page caching
    - Keeping real time statistics
* Suppose we have web application which use Redis to save cached pages. Key will be page hash and value will be serialized page.
* Get page of cache instead dynamically generating it.
* Redis is useful in task queue, LRU cache, distributed locks
* Allocate max 1 GB ram to redis, redis will keep latest 1 GB of data.
* Count unique thing real time.
* Redis can not have secondary index, Cassandra has it
* Redis does not support SQL query syntax, Cassandra does
* Redis is not scalable data store.
* Cassandra is row store, redis is key-value store
* Use it with data you do not mind losing
* It is not a database, it is just key value store.

* userIdD : [tweetID1, tweetID2]
* Timeline service of twitter use it.
* Ziplists compress the list.
* Insertion in Ziplist is slow, so twitter made list of ziplists

#### Redis and Scalability
* Range partitioning: User record number 1-10000 goes to machine 1, 10001=20000 goes to 2.
* Hash partitioning: Using hash function store key to specific machine.
* Replication: Master slave mode, CLients write to master, master replicates to slave and client read from slaves to scale up read performance.

### Redis Command
* `BGSAVE`: Save database to disk in background
* `HSET` : set field in hash
* `APPEND` : Concatenate given string with existing.
* `GET`,`SET`,`INCR`,`DECR`,`EXISTS`,`DEL`,`HGET`,`KEYS`,`HDEL`,`SADD`,`SMEMBERS`,`SREM`
* `PUBLISH`,`SUBSCRIBE` : Client can subscribe to certain key and when that key's value changed that client will be notified
* 

### Redis CLI
* Start redis server using redis-server
* In other terminal,

![](images/reddis_cli.PNG)

* `info` command gives comprehensive view od database state

* There is fast parser between Python and Redis hiredis

In [12]:
import redis

In [13]:
r = redis.StrictRedis()

#### String
    - Basic data type
    - Everything turns to string
    - Max size of string is 512 MB

In [14]:
r.set('name', 'purvil')

True

In [15]:
r.get('name')

b'purvil'

In [16]:
type(r.get('name'))

bytes

In [17]:
r.get('name').decode('utf-8')

'purvil'

### Hash
* As a value we will have hash map (like python dict)

In [18]:
r.hset('users', 'name', 'purvil')

0

In [19]:
r.hmset('users', {'age':26, 'height':195})

True

In [20]:
r.hgetall('users')

{b'name': b'purvil', b'age': b'26', b'height': b'195'}

In [21]:
r.hincrby('users', 'height', 2)

197

In [22]:
r.hgetall('users')

{b'name': b'purvil', b'age': b'26', b'height': b'197'}

In [46]:
r.hget('users', 'age')

b'26'

In [49]:
r.hget('users', 'money') is None # not available key

True

In [50]:
r.hkeys("users")

[b'name', b'age', b'height']

### Configuration

In [23]:
r.config_get('*')

{'dir': 'C:\\redis-2.4.5\\64bit',
 'dbfilename': 'dump.rdb',
 'requirepass': None,
 'masterauth': None,
 'maxmemory': '0',
 'maxmemory-policy': 'volatile-lru',
 'maxmemory-samples': '3',
 'timeout': '0',
 'appendonly': 'no',
 'no-appendfsync-on-rewrite': 'no',
 'appendfsync': 'everysec',
 'save': '3600 1 300 100 60 10000',
 'auto-aof-rewrite-percentage': '100',
 'auto-aof-rewrite-min-size': '1048576',
 'slave-serve-stale-data': 'yes',
 'hash-max-zipmap-entries': '512',
 'hash-max-zipmap-value': '64',
 'list-max-ziplist-entries': '512',
 'list-max-ziplist-value': '64',
 'set-max-intset-entries': '512',
 'zset-max-ziplist-entries': '128',
 'zset-max-ziplist-value': '64',
 'slowlog-log-slower-than': '10000',
 'slowlog-max-len': '64',
 'loglevel': 'verbose'}

In [25]:
# r.config_set('maxclients', 4) # max parallel clients

In [28]:
r.mset({'purvil':'dave', 'japan':'dave', 'bhavika':'joshi'})

True

In [29]:
r.get('japan')

b'dave'

In [30]:
r.getrange('purvil', 0, 2)

b'dav'

* List
    - Implemented as 2 way connected linked list.
    * Constant time to push to/pop from head and tail
    * Linear time to index access
    * Excellent stack/queue basis
    * Useful for distributed task queue.

In [32]:
r.rpush('python:version', '3.5')

1

In [36]:
r.rpush('python:version', '3.6')

2

In [37]:
r.rpush('python:version', '3.7')

3

In [38]:
r.lpush('python:version', '2.7')

4

In [39]:
r.lrange('python:version', 0, -1)

[b'2.7', b'3.5', b'3.6', b'3.7']

In [40]:
r.rpop('python:version')

b'3.7'

In [41]:
r.lpop('python:version')

b'2.7'

In [43]:
r.lrange('python:version',0, -1)

[b'3.5', b'3.6']

In [45]:
r.ltrim('python:version', 0, 0) # index bigger than 0 will be discarded

True

* Sets:
    - Unordered unique elements
    - Constant time membership checkup
    - Set operation like union, intersection ion linear time

In [51]:
r.sadd('animal', 'cats', 'dog', 'cow')

3

In [53]:
r.smembers('animal')

{b'cats', b'cow', b'dog'}

In [54]:
r.sismember('animal', 'monkey')

False

In [57]:
r.sadd('animal_2', 'cats', 'monkey')

1

In [58]:
r.sinter('animal', 'animal_2')

{b'cats'}

In [59]:
r.scard('animal') # Cardinality

3

* Sorted Sets
    - Unique value stored
    - Each value has score
    - Element ordered by score then lexicographically
    - Output n top scored values
    - Filter value by score
    - Retrieve element rank
    - Ranking operation cost log(N) and element insertion cost log(N)

In [62]:
r.zadd('gamers', 119, 'snake')

AttributeError: 'int' object has no attribute 'items'